本教程介绍了如何使用Estimators解决TensorFlow中的虹膜分类问题。Estimator是TensorFlow完整模型的高级表示，其设计目的是易于缩放和异步训练。有关更多详细信息，请参见 Estimators。

请注意，在TensorFlow 2.0中，Keras API可以完成许多相同的任务，并且被认为是更容易学习的API。如果您是从头开始，我们建议您从Keras开始。有关TensorFlow 2.0中可用的高级API的更多信息，请参阅《在Keras上标准化》。

第一件事

为了开始，您将首先导入TensorFlow和您需要的许多库。

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

数据集
本文档中的示例程序构建并测试了一个模型，该模型根据鸢尾花的萼片和 花瓣的大小将其分为三个不同的物种 。

您将使用虹膜数据集训练模型。虹膜数据集包含四个要素和一个 标签。这四个特征确定了单个鸢尾花的以下植物学特征：

* 萼片长度
* 萼片宽度
* 花瓣长度
* 花瓣宽度

根据此信息，您可以定义一些有用的常量来解析数据：

In [2]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

接下来，使用Keras和Pandas下载并解析Iris数据集。请注意，您保留了不同的数据集以进行培训和测试。

In [3]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


您可以检查数据以查看具有四个float功能列和一个int32标签。

In [4]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


对于每个数据集，请分割标签，模型将被训练以进行预测。

In [5]:
train_y = train.pop('Species')
test_y = test.pop('Species')

# The label column has now been removed from the features.
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


估计器编程概述<br>
现在您已经设置了数据，您可以使用TensorFlow Estimator定义模型。估算器是从派生的任何类tf.estimator.Estimator。TensorFlow提供了tf.estimator （例如LinearRegressor）的集合 来实现常见的ML算法。除此以外，您还可以编写自己的 自定义估算器。我们建议刚开始时使用预制的估算器。

要基于预制的Estimators编写TensorFlow程序，您必须执行以下任务：

* 创建一个或多个输入函数。
* 定义模型的特征列。
* 实例化一个估计器，指定要素列和各种超参数。
* 在Estimator对象上调用一个或多个方法，并传递适当的输入函数作为数据源。

让我们看看如何为虹膜分类执行这些任务。

创建输入功能<br>
您必须创建输入函数以提供用于训练，评估和预测的数据。

一个输入功能是返回的函数tf.data.Dataset，输出以下两个元素的元组对象：

* features -Python字典，其中：
    * 每个键都是功能的名称。
    * 每个值都是一个包含该要素所有值的数组。
* label-包含每个示例的标签值的数组 。

只是为了演示输入函数的格式，下面是一个简单的实现：

In [6]:
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
                'SepalWidth':  np.array([2.8, 2.3]),
                'PetalLength': np.array([5.6, 3.3]),
                'PetalWidth':  np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels

您的输入函数可以生成features字典并label以您喜欢的任何方式列出。但是，我们建议使用TensorFlow的Dataset API，它可以解析各种数据。

Dataset API可以为您处理许多常见情况。例如，使用Dataset API，您可以轻松并行地从大量文件中读取记录，并将它们加入单个流中。

为了使示例简单，您将使用pandas加载数据 ，并根据此内存数据构建输入管道：

In [7]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

定义特征列
一个特征柱 是描述如何模型应该从特征字典使用原始输入数据的对象。构建Estimator模型时，会向其传递一系列功能列，这些列描述了您希望模型使用的每个功能。该tf.feature_column模块提供了许多用于表示模型数据的选项。

对于Iris，这4个原始特征是数值，因此我们将构建一个特征列列表，以告知Estimator模型将这四个特征中的每一个表示为32位浮点值。因此，用于创建功能部件列的代码为：

In [8]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

功能列可能比我们在此处显示的功能列复杂得多。您可以在本指南中阅读有关功能列的更多信息。

现在，您已经有了关于如何使模型表示原始特征的描述，您可以构建估计器。

实例化估算器
虹膜问题是经典的分类问题。幸运的是，TensorFlow提供了几个预制的分类器估计器，包括：

* tf.estimator.DNNClassifier 适用于执行多类别分类的深层模型。
* tf.estimator.DNNLinearCombinedClassifier 适用于宽模型和深模型。
* tf.estimator.LinearClassifier 用于基于线性模型的分类器。

对于虹膜问题，tf.estimator.DNNClassifier似乎是最佳选择。实例化此估算器的方法如下：

In [9]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\30660\\AppData\\Local\\Temp\\tmpntldugyk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021EB102F748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


训练，评估和预测
现在有了Estimator对象，您可以调用方法来执行以下操作：

* 训练模型。
* 评估训练后的模型。
* 使用训练有素的模型进行预测。

训练模型<br>
通过调用Estimator的train方法训练模型，如下所示：

In [10]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

请注意，您将input_fn调用包装在中 lambda 以捕获参数，同时提供了一个不包含参数的输入函数，这是Estimator期望的。该steps参数告诉方法在许多训练步骤后停止训练。

In [11]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-10-15T18:41:06Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\30660\AppData\Local\Temp\tmpntldugyk\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-10-15-18:41:06
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.96666664, average_loss = 0.52

评估训练好的模型<br>
现在已经对模型进行了训练，您可以获取有关其性能的一些统计信息。以下代码块根据测试数据评估训练模型的准确性：

根据训练后的模型进行预测（推断）

您现在拥有一个训练有素的模型，该模型可以产生良好的评估结果。现在，您可以使用训练有素的模型，根据一些未标记的测量值来预测鸢尾花的种类。与培训和评估一样，您可以使用单个函数调用进行预测：

In [12]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

该predict方法返回Python迭代的，为每个示例生成预测结果字典。以下代码显示了一些预测及其概率：

In [13]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\30660\AppData\Local\Temp\tmpntldugyk\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (67.7%), expected "Setosa"
Prediction is "Versicolor" (50.2%), expected "Versicolor"
Prediction is "Virginica" (55.7%), expected "Virginica"
